In [2]:
import pandas as pd
from safegraph_py_functions import safegraph_py_functions as sgpy
import os
from dotenv import load_dotenv, find_dotenv

In [3]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)
root_dir = os.environ.get("ROOT_DIR")
raw_data_dir = os.path.join(root_dir,'data/raw')

In [4]:
# local directory where we want to put all the data
patterns_path = os.path.join(raw_data_dir,'monthly-patterns')
# print(local)
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(patterns_path):
    for file in f:
        if file.endswith('.csv.gz') and 'patterns-part' in file:
            files.append(os.path.join(patterns_path, r, file))

In [5]:
philly_zips = pd.Series(['19102', '19103', '19104', '19106', '19107', '19109', '19111', '19112', '19114', '19115', 
               '19116', '19118', '19119', '19120', '19121', '19122', '19123', '19124', '19125', '19126', 
               '19127', '19128', '19129', '19130', '19131', '19132', '19133', '19134', '19135', '19136', 
               '19137', '19138', '19139', '19140', '19141', '19142', '19143', '19144', '19145', '19146', 
               '19147', '19148', '19149', '19150', '19151', '19152', '19153', '19154'], name = 'postal_code')

philly_zips = philly_zips.to_frame()



In [6]:
# read in a csv with philly zipcodes
# philly_zips = pd.read_csv()

In [10]:
# columns that we keep and aggregate
keep_cols = ['postal_code', 'safegraph_place_id', 'date_range_start', 'visits_by_day']
keep_cols_2 = ['postal_code', 'date','day_visit_counts']
group_by_cols = ['postal_code', 'date']

def filter_and_explode(df):
    df = df[keep_cols]
    df['postal_code'] = df['postal_code'].apply(lambda x: ('00000' + str(x))[-5:])
    df = philly_zips.merge(df)
    df = sgpy.explode_json_array(
        df, array_column ='visits_by_day', value_col_name='day_visit_counts', 
        place_key='safegraph_place_id', file_key='date_range_start', array_sequence='day', 
        keep_index=False, zero_index=False)
    df['date_range_start'] = pd.to_datetime(df['date_range_start'])
    temp = df['day'].apply(lambda x: pd.Timedelta(x-1, unit='D'))
    df['date'] = df['date_range_start'] + temp
    df = df[keep_cols_2].groupby(group_by_cols).agg('sum').reset_index()
    return df

In [13]:
philly_patterns = [filter_and_explode(pd.read_csv(file)) for file in files]
philly_patterns_df = pd.concat(philly_patterns)

//anaconda3/envs/musa-620/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
processed_data_dir = os.path.join(root_dir,'data/processed')
philly_patterns_df.to_csv(os.path.join(processed_data_dir,'philly_patterns_by_zip.csv'), index = False)